In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
pd.options.display.float_format = '{:.2f}'.format

In [2]:
# Read in New York Times online csv file with daily COVID data
# Convert FIPS number to integer, convert date to datetime, save file to local csv

us_covid_df = pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv')
us_covid_df['fips'] = us_covid_df['fips'].fillna(0).astype(np.int64)
us_covid_df['date'] = pd.to_datetime(us_covid_df['date'], yearfirst=True)
us_covid_df.to_csv('us_covid.csv', index=False)

In [3]:
# Read in national population data provided by state, county, FIPS

us_pops_df = pd.read_csv('US_Populations.csv', engine='python', thousands=',')
us_pops_df

,FIPStxt,State,State2,Area_Name,Region,population
0,2000,Alaska,AK,Alaska,State,731545
1,2013,Alaska,AK,Aleutians East Borough,County,3337
2,2016,Alaska,AK,Aleutians West Census Area,County,5634
3,2020,Alaska,AK,Anchorage,County,288000
4,2050,Alaska,AK,Bethel Census Area,County,18386
...,...,...,...,...,...,...
3146,22119,Louisiana,LA,Webster,County,38340
3147,22121,Louisiana,LA,West Baton Rouge,County,26465
3148,22123,Louisiana,LA,West Carroll,County,10830
3149,22125,Louisiana,LA,West Feliciana,County,15568


In [4]:
# Merge New York Times data with population data

us_data_df = pd.merge(us_covid_df, us_pops_df, left_on=['county', 'state'], right_on=['Area_Name', 'State'])
us_data_df

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
0,2020-01-21,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
1,2020-01-22,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
2,2020-01-23,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
3,2020-01-24,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
4,2020-01-25,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
...,...,...,...,...,...,...,...,...,...,...,...,...
1475561,2021-07-18,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1475562,2021-07-19,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1475563,2021-07-20,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1475564,2021-07-21,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86


In [5]:
# Separate out data from the state level

states = us_data_df[(us_data_df['Region'] == 'State')]
states

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
38871,2020-03-07,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
38873,2020-03-08,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
38875,2020-03-09,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
38877,2020-03-10,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
38879,2020-03-11,District of Columbia,District of Columbia,11001,10,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
...,...,...,...,...,...,...,...,...,...,...,...,...
902784,2021-07-18,Arkansas,Arkansas,5001,2358,40.00,5000,Arkansas,AR,Arkansas,State,3017804
902786,2021-07-19,Arkansas,Arkansas,5001,2397,39.00,5000,Arkansas,AR,Arkansas,State,3017804
902788,2021-07-20,Arkansas,Arkansas,5001,2412,39.00,5000,Arkansas,AR,Arkansas,State,3017804
902790,2021-07-21,Arkansas,Arkansas,5001,2418,39.00,5000,Arkansas,AR,Arkansas,State,3017804


In [6]:
# Separate out data specific to county level, drop redundant columns
#  Create infection and death rates, updating for each date in the dataframe

counties = us_data_df.drop(['fips', 'State','Area_Name'], axis=1)
counties['current_crate'] = counties['cases']*100000/counties['population']
counties['current_drate'] = counties['deaths']*100000/counties['population']
counties = counties[(counties['Region'] == 'County')]
counties

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
...,...,...,...,...,...,...,...,...,...,...,...
1475561,2021-07-18,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1475562,2021-07-19,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1475563,2021-07-20,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1475564,2021-07-21,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00


In [7]:
# Isolate data from most current day.

counties_us = counties[(counties['date'] == '2021-07-22') & (counties['Region'] == 'County')]
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
548,2021-07-22,Snohomish,Washington,41695,614.00,53061,WA,County,822083,5071.87,74.69
1094,2021-07-22,Cook,Illinois,560675,11107.00,17031,IL,County,5150233,10886.40,215.66
1639,2021-07-22,Orange,California,278209,5139.00,6059,CA,County,3175692,8760.58,161.82
2183,2021-07-22,Maricopa,Arizona,572062,10403.00,4013,AZ,County,4485414,12753.83,231.93
2727,2021-07-22,Los Angeles,California,1276818,24609.00,6037,CA,County,10039107,12718.44,245.13
...,...,...,...,...,...,...,...,...,...,...,...
1474557,2021-07-22,Wheeler,Oregon,35,1.00,41069,OR,County,1332,2627.63,75.08
1474840,2021-07-22,King,Texas,11,0.00,48269,TX,County,272,4044.12,0.00
1475092,2021-07-22,Esmeralda,Nevada,40,0.00,32009,NV,County,873,4581.90,0.00
1475340,2021-07-22,Loving,Texas,3,0.00,48301,TX,County,169,1775.15,0.00


In [8]:
# Create up to date rates so rankings can be performed using the latest data.

counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)
counties_us = counties_us[(counties_us['Region'] == 'County')]
counties_us

<ipython-input-8-608f6ade14d5>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
<ipython-input-8-608f6ade14d5>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate
548,2021-07-22,Snohomish,Washington,41695,614.00,53061,WA,County,822083,5071.87,74.69,74.69,5071.87
1094,2021-07-22,Cook,Illinois,560675,11107.00,17031,IL,County,5150233,10886.40,215.66,215.66,10886.40
1639,2021-07-22,Orange,California,278209,5139.00,6059,CA,County,3175692,8760.58,161.82,161.82,8760.58
2183,2021-07-22,Maricopa,Arizona,572062,10403.00,4013,AZ,County,4485414,12753.83,231.93,231.93,12753.83
2727,2021-07-22,Los Angeles,California,1276818,24609.00,6037,CA,County,10039107,12718.44,245.13,245.13,12718.44
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1474557,2021-07-22,Wheeler,Oregon,35,1.00,41069,OR,County,1332,2627.63,75.08,75.08,2627.63
1474840,2021-07-22,King,Texas,11,0.00,48269,TX,County,272,4044.12,0.00,0.00,4044.12
1475092,2021-07-22,Esmeralda,Nevada,40,0.00,32009,NV,County,873,4581.90,0.00,0.00,4581.90
1475340,2021-07-22,Loving,Texas,3,0.00,48301,TX,County,169,1775.15,0.00,0.00,1775.15


In [9]:
# Create ranking of infection rates by county for each state

counties_us.sort_values(by=['state', 'crate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_case']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case
944155,2021-07-22,Hale,Alabama,2305,78.00,1065,AL,County,14651,15732.71,532.39,532.39,15732.71,1
1028075,2021-07-22,Clarke,Alabama,3615,62.00,1025,AL,County,23622,15303.53,262.47,262.47,15303.53,2
738081,2021-07-22,Lowndes,Alabama,1438,54.00,1085,AL,County,9726,14785.11,555.21,555.21,14785.11,3
654182,2021-07-22,Etowah,Alabama,14563,368.00,1055,AL,County,102268,14240.04,359.84,359.84,14240.04,4
548646,2021-07-22,Franklin,Alabama,4397,81.00,1059,AL,County,31362,14020.15,258.27,258.27,14020.15,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
901349,2021-07-22,Sublette,Wyoming,833,7.00,56035,WY,County,9831,8473.20,71.20,71.20,8473.20,19
900867,2021-07-22,Converse,Wyoming,1096,19.00,56009,WY,County,13822,7929.39,137.46,137.46,7929.39,20
1121092,2021-07-22,Lincoln,Wyoming,1486,14.00,56023,WY,County,19830,7493.70,70.60,70.60,7493.70,21
1248618,2021-07-22,Niobrara,Wyoming,164,3.00,56027,WY,County,2356,6960.95,127.33,127.33,6960.95,22


In [10]:
# Create ranking of death rates by county for each state

counties_us.sort_values(by=['state', 'drate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_death']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case,state_death
738081,2021-07-22,Lowndes,Alabama,1438,54.00,1085,AL,County,9726,14785.11,555.21,555.21,14785.11,3,1
944155,2021-07-22,Hale,Alabama,2305,78.00,1065,AL,County,14651,15732.71,532.39,532.39,15732.71,1,2
313157,2021-07-22,Walker,Alabama,7545,286.00,1127,AL,County,63521,11877.96,450.24,450.24,11877.96,29,3
652242,2021-07-22,Clay,Alabama,1618,59.00,1027,AL,County,13235,12225.16,445.79,445.79,12225.16,23,4
653697,2021-07-22,Dallas,Alabama,3674,163.00,1047,AL,County,37196,9877.41,438.22,438.22,9877.41,55,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1121092,2021-07-22,Lincoln,Wyoming,1486,14.00,56023,WY,County,19830,7493.70,70.60,70.60,7493.70,21,19
794225,2021-07-22,Hot Springs,Wyoming,385,3.00,56017,WY,County,4413,8724.22,67.98,67.98,8724.22,17,20
1093008,2021-07-22,Uinta,Wyoming,2368,13.00,56041,WY,County,20226,11707.70,64.27,64.27,11707.70,5,21
311684,2021-07-22,Teton,Wyoming,3865,11.00,56039,WY,County,23464,16472.04,46.88,46.88,16472.04,1,22


In [11]:
# Merge data from today with historical data to provide current rates + rates from previous points in time.

usa = pd.merge(counties, counties_us[['state', 'county','drate', 'crate', 'state_death', 'state_case']], on=['state', 'county'])
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,74.69,5071.87,21,25
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,74.69,5071.87,21,25
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,74.69,5071.87,21,25
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,74.69,5071.87,21,25
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,74.69,5071.87,21,25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1472124,2021-07-18,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4
1472125,2021-07-19,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4
1472126,2021-07-20,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4
1472127,2021-07-21,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4


In [12]:
# Break out daily case data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_case', 'date'], inplace=True)
usa['case_day'] = usa.groupby(['state','county']).cases.diff().fillna(usa.cases)
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day
940239,2020-03-30,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,532.39,15732.71,2,1,1.00
940240,2020-03-31,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,532.39,15732.71,2,1,0.00
940241,2020-04-01,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,532.39,15732.71,2,1,0.00
940242,2020-04-02,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,532.39,15732.71,2,1,0.00
940243,2020-04-03,Hale,Alabama,3,0.00,1065,AL,County,14651,20.48,0.00,532.39,15732.71,2,1,2.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1139976,2021-07-18,Crook,Wyoming,467,13.00,56011,WY,County,7584,6157.70,171.41,171.41,6184.07,6,23,0.00
1139977,2021-07-19,Crook,Wyoming,468,13.00,56011,WY,County,7584,6170.89,171.41,171.41,6184.07,6,23,1.00
1139978,2021-07-20,Crook,Wyoming,469,13.00,56011,WY,County,7584,6184.07,171.41,171.41,6184.07,6,23,1.00
1139979,2021-07-21,Crook,Wyoming,469,13.00,56011,WY,County,7584,6184.07,171.41,171.41,6184.07,6,23,0.00


In [13]:
# Break out daily death data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_death', 'date'], inplace=True)
usa['death_day'] = usa.groupby(['state','county']).deaths.diff().fillna(usa.deaths)
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
735125,2020-03-26,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,555.21,14785.11,1,3,1.00,0.00
735126,2020-03-27,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,555.21,14785.11,1,3,0.00,0.00
735127,2020-03-28,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,555.21,14785.11,1,3,0.00,0.00
735128,2020-03-29,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,555.21,14785.11,1,3,0.00,0.00
735129,2020-03-30,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,555.21,14785.11,1,3,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
732700,2021-07-18,Albany,Wyoming,4697,17.00,56001,WY,County,38880,12080.76,43.72,46.30,12119.34,23,3,0.00,0.00
732701,2021-07-19,Albany,Wyoming,4706,17.00,56001,WY,County,38880,12103.91,43.72,46.30,12119.34,23,3,9.00,0.00
732702,2021-07-20,Albany,Wyoming,4709,18.00,56001,WY,County,38880,12111.63,46.30,46.30,12119.34,23,3,3.00,1.00
732703,2021-07-21,Albany,Wyoming,4711,18.00,56001,WY,County,38880,12116.77,46.30,46.30,12119.34,23,3,2.00,0.00


In [14]:
# Export updated national data to csv for use in Tableau

usa.to_csv('daily_covid_us.csv', index=False)

In [15]:
# Filter California data to show only numbers for today

counties_cal = usa[(usa['date'] == '2021-07-22') & (usa['state'] == 'California')]
counties_cal

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
374009,2021-07-22,Imperial,California,28964,741.00,6025,CA,County,181215,15983.22,408.91,408.91,15983.22,1,2,31.00,0.00
2727,2021-07-22,Los Angeles,California,1276818,24609.00,6037,CA,County,10039107,12718.44,245.13,245.13,12718.44,2,5,2747.00,6.00
185905,2021-07-22,San Bernardino,California,303673,5230.00,6071,CA,County,2180085,13929.41,239.90,239.90,13929.41,3,4,323.00,-1.00
656074,2021-07-22,Inyo,California,1446,38.00,6027,CA,County,18039,8015.97,210.65,210.65,8015.97,4,25,0.00,0.00
82973,2021-07-22,Stanislaus,California,64912,1084.00,6099,CA,County,550660,11788.04,196.85,196.85,11788.04,5,9,120.00,1.00
65975,2021-07-22,San Joaquin,California,76034,1446.00,6077,CA,County,762148,9976.28,189.73,189.73,9976.28,6,17,0.00,0.00
38367,2021-07-22,Riverside,California,304512,4657.00,6065,CA,County,2470546,12325.70,188.50,188.50,12325.70,7,7,452.00,0.00
102930,2021-07-22,Tulare,California,50074,854.00,6107,CA,County,466195,10741.00,183.19,183.19,10741.00,8,11,0.00,0.00
594369,2021-07-22,Merced,California,32814,484.00,6047,CA,County,277680,11817.20,174.30,174.30,11817.20,9,8,0.00,0.00
37361,2021-07-22,Fresno,California,103504,1736.00,6019,CA,County,999101,10359.71,173.76,173.76,10359.71,10,14,66.00,0.00


In [16]:
# Show top 10 California counties based on largest current infection rate

counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)
top10_ca_case_rates = counties_cal.head(10)
top10_ca_case_rates

<ipython-input-16-94608615179b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
1394422,2021-07-22,Lassen,California,5866,26.00,6035,CA,County,30573,19186.86,85.04,85.04,19186.86,35,1,3.00,0.00
374009,2021-07-22,Imperial,California,28964,741.00,6025,CA,County,181215,15983.22,408.91,408.91,15983.22,1,2,31.00,0.00
796100,2021-07-22,Kings,California,23366,248.00,6031,CA,County,152940,15277.89,162.16,162.16,15277.89,11,3,81.00,0.00
185905,2021-07-22,San Bernardino,California,303673,5230.00,6071,CA,County,2180085,13929.41,239.90,239.90,13929.41,3,4,323.00,-1.00
2727,2021-07-22,Los Angeles,California,1276818,24609.00,6037,CA,County,10039107,12718.44,245.13,245.13,12718.44,2,5,2747.00,6.00
233865,2021-07-22,Kern,California,112063,1416.00,6029,CA,County,900202,12448.65,157.30,157.30,12448.65,14,6,0.00,0.00
38367,2021-07-22,Riverside,California,304512,4657.00,6065,CA,County,2470546,12325.70,188.50,188.50,12325.70,7,7,452.00,0.00
594369,2021-07-22,Merced,California,32814,484.00,6047,CA,County,277680,11817.20,174.30,174.30,11817.20,9,8,0.00,0.00
82973,2021-07-22,Stanislaus,California,64912,1084.00,6099,CA,County,550660,11788.04,196.85,196.85,11788.04,5,9,120.00,1.00
795617,2021-07-22,Colusa,California,2322,18.00,6011,CA,County,21547,10776.44,83.54,83.54,10776.44,36,10,0.00,0.00


In [17]:
# Show top 10 California counties based on largest current death rate

counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)
top10_ca_death_rates = counties_cal.head(10)
top10_ca_death_rates

<ipython-input-17-018d0b919d54>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
374009,2021-07-22,Imperial,California,28964,741.00,6025,CA,County,181215,15983.22,408.91,408.91,15983.22,1,2,31.00,0.00
2727,2021-07-22,Los Angeles,California,1276818,24609.00,6037,CA,County,10039107,12718.44,245.13,245.13,12718.44,2,5,2747.00,6.00
185905,2021-07-22,San Bernardino,California,303673,5230.00,6071,CA,County,2180085,13929.41,239.90,239.90,13929.41,3,4,323.00,-1.00
656074,2021-07-22,Inyo,California,1446,38.00,6027,CA,County,18039,8015.97,210.65,210.65,8015.97,4,25,0.00,0.00
82973,2021-07-22,Stanislaus,California,64912,1084.00,6099,CA,County,550660,11788.04,196.85,196.85,11788.04,5,9,120.00,1.00
65975,2021-07-22,San Joaquin,California,76034,1446.00,6077,CA,County,762148,9976.28,189.73,189.73,9976.28,6,17,0.00,0.00
38367,2021-07-22,Riverside,California,304512,4657.00,6065,CA,County,2470546,12325.70,188.50,188.50,12325.70,7,7,452.00,0.00
102930,2021-07-22,Tulare,California,50074,854.00,6107,CA,County,466195,10741.00,183.19,183.19,10741.00,8,11,0.00,0.00
594369,2021-07-22,Merced,California,32814,484.00,6047,CA,County,277680,11817.20,174.30,174.30,11817.20,9,8,0.00,0.00
37361,2021-07-22,Fresno,California,103504,1736.00,6019,CA,County,999101,10359.71,173.76,173.76,10359.71,10,14,66.00,0.00


In [18]:
# Create dataframe with historical data for Top 10 counties based on current case rate

top10c_df = top10_ca_case_rates[['county', 'state']]
top10_ca_case_df = pd.merge(usa, top10c_df, on=['county', 'state'])
top10_ca_case_df['date'] = pd.to_datetime(top10_ca_case_df['date']).dt.strftime('%m/%d/%y')
top10_ca_case_df = top10_ca_case_df[['date', 'county','cases', 'deaths', 'current_crate', 'current_drate', 'state_death', 'state_case', 'case_day', 'death_day']]
top10_ca_case_df

,date,county,cases,deaths,current_crate,current_drate,state_death,state_case,case_day,death_day
0,03/20/20,Imperial,4,0.00,2.21,0.00,1,2,4.00,0.00
1,03/21/20,Imperial,4,0.00,2.21,0.00,1,2,0.00,0.00
2,03/22/20,Imperial,4,0.00,2.21,0.00,1,2,0.00,0.00
3,03/23/20,Imperial,4,0.00,2.21,0.00,1,2,0.00,0.00
4,03/24/20,Imperial,9,0.00,4.97,0.00,1,2,5.00,0.00
...,...,...,...,...,...,...,...,...,...,...
4897,07/18/21,Colusa,2322,18.00,10776.44,83.54,36,10,0.00,0.00
4898,07/19/21,Colusa,2322,18.00,10776.44,83.54,36,10,0.00,0.00
4899,07/20/21,Colusa,2322,18.00,10776.44,83.54,36,10,0.00,0.00
4900,07/21/21,Colusa,2322,18.00,10776.44,83.54,36,10,0.00,0.00


In [19]:
# Create dataframe with historical data for Top 10 counties based on current death rate

top10d_df = top10_ca_death_rates[['county', 'state']]
top10_ca_death_df = pd.merge(usa, top10d_df, on=['county', 'state'])
top10_ca_death_df['date'] = pd.to_datetime(top10_ca_death_df['date']).dt.strftime('%m/%d/%y')
top10_ca_death_df = top10_ca_death_df[['date', 'county','cases', 'deaths', 'current_crate', 'current_drate', 'state_death', 'state_case', 'case_day', 'death_day']]
top10_ca_death_df

,date,county,cases,deaths,current_crate,current_drate,state_death,state_case,case_day,death_day
0,03/20/20,Imperial,4,0.00,2.21,0.00,1,2,4.00,0.00
1,03/21/20,Imperial,4,0.00,2.21,0.00,1,2,0.00,0.00
2,03/22/20,Imperial,4,0.00,2.21,0.00,1,2,0.00,0.00
3,03/23/20,Imperial,4,0.00,2.21,0.00,1,2,0.00,0.00
4,03/24/20,Imperial,9,0.00,4.97,0.00,1,2,5.00,0.00
...,...,...,...,...,...,...,...,...,...,...
4998,07/18/21,Fresno,103055,1730.00,10314.77,173.16,10,14,0.00,0.00
4999,07/19/21,Fresno,103298,1731.00,10339.09,173.26,10,14,243.00,1.00
5000,07/20/21,Fresno,103350,1731.00,10344.30,173.26,10,14,52.00,0.00
5001,07/21/21,Fresno,103438,1736.00,10353.11,173.76,10,14,88.00,5.00


In [20]:
# Export California Top 10 case data to csv

top10_ca_case_df.to_csv('top10cases.csv', float_format='%.2f', index=False)

In [21]:
# Export California Top 10 death data to csv

top10_ca_death_df.to_csv('top10deaths.csv', float_format='%.2f', index=False)